# Sim Network

In [13]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import combinations

In [5]:
class cfg:
    TRAINING_DATA_PATH = './data/train.csv'

## Prepare data

In [18]:
df = pd.read_csv(cfg.TRAINING_DATA_PATH)
df = df[['id', 'name', 'latitude', 'longitude', 'city', 'state', 'zip', 'country', 'categories', 'point_of_interest']]
df = df[10000:20000]
df.head()

,id,name,latitude,longitude,city,state,zip,country,categories,point_of_interest
10000,E_0246d8074db266,LaFee Beach Club,39.583069,26.914058,Balikesir,Edremit,NaN,TR,"Nightclubs, Music Venues",P_91fe1a75b28452
10001,E_0246ea7e1057ab,Consultorio Odontológico,-25.356974,-57.505734,San Lorenzo,Central,NaN,PY,Dentist's Offices,P_d87a6bd4a758ea
10002,E_024708969f3193,Oak Knoll Park Baseball,33.820926,-118.033060,Cypress,CA,90630,US,Baseball Fields,P_b38f74f6ea56ff
10003,E_02471989168598,Mavi Köşe,36.588993,36.168899,İskenderun,Türkiye,31200,TR,"Fast Food Restaurants, Burger Joints",P_050caeb0086a49
10004,E_0247228a7f0483,Холмогорова 30,56.874012,53.217625,NaN,NaN,NaN,RU,Residential Buildings (Apartments / Condos),P_311d6daf9a16b1


In [68]:
pair1_cols = ['id1', 'name1', 'latitude1', 'longitude1', 'city1', 'state1', 'zip1', 'country1', 'categories1']
pair2_cols = ['id2', 'name2', 'latitude2', 'longitude2', 'city2', 'state2', 'zip2', 'country2', 'categories2']
pair_df = pd.DataFrame(columns=pair1_cols + pair2_cols)
new_pair = pair_df.copy(deep=True)
pair_df

,id1,name1,latitude1,longitude1,city1,state1,zip1,country1,categories1,id2,name2,latitude2,longitude2,city2,state2,zip2,country2,categories2


In [73]:
counter = 0
for poi, poi_df in tqdm(df.groupby('point_of_interest')):
    if len(poi_df) > 2:
        new_pair = new_pair[0:0]
        poi_df = poi_df.drop(columns=['point_of_interest'])
        match_combs = np.array(list(combinations(poi_df['id'], 2)))
        pair1 = poi_df[poi_df['id'].isin(match_combs[:, 0])]
        pair2 = poi_df[poi_df['id'].isin(match_combs[:, 1])]
        new_pair[pair1_cols] = pair1
        new_pair[pair2_cols] = pair2
        pair_df = pd.concat([pair_df, new_pair], ignore_index=True)
print(pair_df)

100%|██████████| 9920/9920 [00:00<00:00, 80784.98it/s]

                 id1                                   name1  latitude1  \
0   E_0262f091d68e6e             Lowes Royal Pacific Rm 2145  28.468909   
1   E_03d00064135f8b              Loews Royal Pacific Resort  28.469941   
2   E_0289d81e89f97e  Ankara Şehirler Arası Otobüs Terminali  39.918160   
3   E_02b4afac7935be                                    Aşti  39.918496   
4   E_043d6c4bf773c0                             aşti otogar  39.919267   
5   E_0323b4731f9db0                             Pantai Kuta  -8.792904   
6   E_0386d49ff4c42e                              Kuta Beach  -8.903687   
7   E_0325c015ad13d7                                   Tekne  41.077022   
8   E_03720e9a35bd9a                         İstanbul Boğazi  41.018787   
9   E_024d8a8f6fdbc1                          Estúdios Globo -22.961017   
10  E_0383597fbd1bd3       Cidade Cenografica flor do caribe -22.961193   
11  E_02761e40f68230                   Sukarno Hatta airport  -7.329110   
12  E_0351377fc6b71d     